In [1]:
import csv

from PIL import Image

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [2]:
class NIPA_Statistics(Dataset):
    def __init__(self):
        self.train_img_dir = './train/'
        self.train_label_dir = './train/train.tsv'
        self.test_img_dir = './test/'
        self.test_label_dir = './test/test.tsv'
        
        self.imgs = []
        with open(self.train_label_dir, 'r') as tsvfile:
            for line in csv.reader(tsvfile, delimiter='\t'):
                img, *_ = line
                img = self.train_img_dir + img
                self.imgs.append(img)
        with open(self.test_label_dir, 'r') as tsvfile:
            for line in csv.reader(tsvfile, delimiter='\t'):
                img, *_ = line
                img = self.test_img_dir + img
                self.imgs.append(img)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = Image.open(img_path).convert('RGB')
        return transforms.ToTensor()(img)

    def __len__(self):
        return len(self.imgs)

In [3]:
dataset = NIPA_Statistics()
loader = DataLoader(dataset, batch_size=64, shuffle=False)

mean = 0.
std = 0.
nb_samples = 0.
for data in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
print(mean, std)

tensor([0.4633, 0.4887, 0.4060]) tensor([0.1824, 0.1601, 0.1958])
